In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=21, padding=10)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=15, padding=7)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=11, padding=5)
        
        self.batchnorm32 = nn.BatchNorm1d(num_features=32)
        self.batchnorm64 = nn.BatchNorm1d(num_features=64)
        self.batchnorm128 = nn.BatchNorm1d(num_features=128)
        self.batchnorm512 = nn.BatchNorm1d(num_features=512)
        
        self.fc1 = nn.Linear(4736, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 1)
        
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=2)        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.batchnorm32(x))
        x = self.maxpool(x)
        
        x = self.conv2(x)
        x = F.relu(self.batchnorm64(x))
        x = self.maxpool(x)
        
        x = self.conv3(x)
        x = F.relu(self.batchnorm128(x))
        x = self.maxpool(x)
        
        # Flatten input for fully connected layers
        x = x.view(x.shape[0], -1) 
        
        x = self.fc1(x)
        x = F.relu(self.batchnorm512(x))
        
        x = self.fc2(x)
        x = F.relu(self.batchnorm512(x))
        
        x = self.fc3(x)
        return x

In [ ]:
import h5py

traindata = h5py.File("../train/scsn_p_2000_2017_6sec_0.5r_pick_train.hdf5")
testdata = h5py.File("../test/scsn_p_2000_2017_6sec_0.5r_pick_test.hdf5")

In [ ]:
from torch.utils.data import DataLoader

batch_size = 500

# Load test data
trainset = traindata["X"]
testset = testdata["X"]
# Zip data and ground truth together for dataloader?

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True)

In [ ]:
model = CNN()

In [ ]:
def parallelize(model):
    device_ids = [i for i in range(torch.cuda.device_count())]
    model = torch.nn.DataParallel(model, device_ids=device_ids)
    return model

In [ ]:
model = parallelize(model)

In [ ]:
from torch import optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 10

for epoch in range(epochs):
    for batch, labels in trainloader:
        # Clear gradients in optimizer
        optimizer.zero_grad()
        # training
    else:
        with torch.no_grad():
            for batch, labels in testloader:
                # validation
                pass
    # Print epoch summary
        